# Capítulo 3: Importação e Exportação de Arquivos CSV

Este notebook demonstra todas as funcionalidades de importação e exportação CSV no DuckDB.

## Setup Inicial

In [ ]:
import duckdb
import os
import tempfile

# Criar diretório temporário para exemplos
temp_dir = tempfile.gettempdir()
print(f"Diretório temporário: {temp_dir}")

## Criar Arquivos CSV de Exemplo

In [ ]:
# CSV 1: Vendas básicas
vendas_csv = os.path.join(temp_dir, 'vendas.csv')
with open(vendas_csv, 'w', encoding='utf-8') as f:
    f.write('produto,quantidade,preco,data\n')
    f.write('Notebook,10,2500.50,2024-01-15\n')
    f.write('Mouse,50,25.00,2024-01-16\n')
    f.write('Teclado,30,150.00,2024-01-17\n')
    f.write('Monitor,15,800.00,2024-01-18\n')

print(f"Arquivo criado: {vendas_csv}")

## 1. Leitura Básica de CSV

In [ ]:
# Criar conexão DuckDB
con = duckdb.connect()

# Leitura direta - auto-detect
result = con.sql(f"SELECT * FROM '{vendas_csv}'")
result.show()

In [ ]:
# Usando read_csv
result = con.sql(f"SELECT * FROM read_csv('{vendas_csv}')")
result.show()

In [ ]:
# Converter para DataFrame Pandas
df = duckdb.read_csv(vendas_csv).df()
df

## 2. Parâmetros Personalizados

In [ ]:
# CSV formato brasileiro
vendas_br_csv = os.path.join(temp_dir, 'vendas_br.csv')
with open(vendas_br_csv, 'w', encoding='utf-8') as f:
    f.write('produto;quantidade;preco;data\n')
    f.write('Notebook;10;2500,50;15/01/2024\n')
    f.write('Mouse;50;25,00;16/01/2024\n')

result = con.sql(f"""
    SELECT * FROM read_csv('{vendas_br_csv}',
        delim = ';',
        decimal_separator = ',',
        dateformat = '%d/%m/%Y'
    )
""")
result.show()

## 3. Criar Tabelas a partir de CSV

In [ ]:
# CREATE TABLE AS (recomendado)
con.sql(f"CREATE OR REPLACE TABLE vendas AS SELECT * FROM '{vendas_csv}'")
con.sql("SELECT * FROM vendas").show()

In [ ]:
# Verificar estrutura da tabela
con.sql("DESCRIBE vendas").show()

## 4. Exportação para CSV

In [ ]:
# Exportar tabela para CSV
output_csv = os.path.join(temp_dir, 'vendas_output.csv')
con.sql(f"COPY vendas TO '{output_csv}'")
print(f"Arquivo exportado: {output_csv}")

In [ ]:
# Exportar resultado de query
output_query = os.path.join(temp_dir, 'vendas_filtradas.csv')
con.sql(f"""
    COPY (
        SELECT produto, quantidade * preco as total
        FROM vendas
        WHERE preco > 100
    ) TO '{output_query}'
""")

# Verificar resultado
con.sql(f"SELECT * FROM '{output_query}'").show()

## 5. Pipeline ETL Completo

In [ ]:
# Pipeline: Ler, Transformar e Salvar
output_pipeline = os.path.join(temp_dir, 'vendas_transformadas.csv')

con.sql(f"""
    COPY (
        SELECT
            produto,
            quantidade,
            preco,
            quantidade * preco as valor_total,
            CASE
                WHEN preco > 500 THEN 'Alto'
                WHEN preco > 100 THEN 'Médio'
                ELSE 'Baixo'
            END as categoria_preco
        FROM '{vendas_csv}'
        ORDER BY valor_total DESC
    ) TO '{output_pipeline}' (HEADER, DELIMITER ',')
""")

# Visualizar resultado
con.sql(f"SELECT * FROM '{output_pipeline}'").show()

## Exercício Prático

In [ ]:
# Criar dados de vendas
vendas_ex = os.path.join(temp_dir, 'sistema_vendas.csv')
with open(vendas_ex, 'w', encoding='utf-8') as f:
    f.write('id,cliente,produto,quantidade,preco_unitario,data\n')
    f.write('1,Alice,Notebook,2,2500.00,2024-01-15\n')
    f.write('2,Bob,Mouse,5,25.00,2024-01-15\n')
    f.write('3,Alice,Teclado,3,150.00,2024-01-16\n')
    f.write('4,Carlos,Monitor,1,800.00,2024-01-16\n')
    f.write('5,Bob,Notebook,1,2500.00,2024-01-17\n')

# Importar
con.sql(f"CREATE OR REPLACE TABLE vendas_ex AS SELECT * FROM '{vendas_ex}'")
con.sql("SELECT * FROM vendas_ex").show()

In [ ]:
# Análise: Total por produto
con.sql("""
    SELECT
        produto,
        sum(quantidade) as qtd_total,
        sum(quantidade * preco_unitario) as valor_total
    FROM vendas_ex
    GROUP BY produto
    ORDER BY valor_total DESC
""").show()

In [ ]:
# Análise: Total por cliente
con.sql("""
    SELECT
        cliente,
        count(*) as num_compras,
        sum(quantidade * preco_unitario) as valor_total
    FROM vendas_ex
    GROUP BY cliente
    ORDER BY valor_total DESC
""").show()

## Cleanup

In [ ]:
con.close()
print("Conexão fechada!")